### 1.Imports

In [1]:
import re ,json , os ,pickle
from Operator import *
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize


### 2.Dataset and Base directory path

In [2]:
DIR_PATH = os.path.dirname(os.path.abspath(""))

DATASET_PATH = os.path.join(DIR_PATH, "DataSet")

STOPWORDBBC_PATH = os.path.join(DATASET_PATH, "stopwordbbc.txt")
STOPWORDTRUMP_PATH = os.path.join(DATASET_PATH, "stopwordtrump.txt")

DATASET_BBCSPORT = os.path.join(DATASET_PATH, "bbcsport")
DATASET_TRUMPSPEECHS = os.path.join(DATASET_PATH, "trumpspeechs")


CODE_PATH = os.path.join(DIR_PATH, "Code")
PICKLES_PATH = os.path.join(DIR_PATH, "Pickles")

### 3.Stop words and Special characters

In [3]:
infile  = open(STOPWORDTRUMP_PATH, "r")


stopwordtrump_list    = word_tokenize(infile.read())
stopwordtrump_list    = [i for i in stopwordtrump_list if i]

infile  = open(STOPWORDBBC_PATH, "r")

stopwordbbc_list    = word_tokenize(infile.read())
stopwordbbc_list    = [i for i in stopwordbbc_list if i]

specialchar_list = ['.',' ',',','[',']','(',')','"',':','?','','-']

### 4. Loading Trump dataset

#### 4.1 Stemming and removing special characters


In [4]:
ps = PorterStemmer()
for r, d, files in os.walk(DATASET_TRUMPSPEECHS):
    speeches = [None for _ in range(len(files))]
    for speech in files:
        speech_no    = int(re.search(r'\d+',speech)[0])
        infile       = open(os.path.join(DATASET_TRUMPSPEECHS,speech), "r")
        content      = infile.read()
        content      = content.split('\n')[1]
        content      = re.sub(r"[^a-zA-Z0-9\']+", ' ', content)
        content_list = word_tokenize(content)
        stem_speech  = [ps.stem(word) for word in content_list]
        speeches[speech_no] = stem_speech
        infile.close()

#### 4.2 Generting Posting lists

In [5]:
all_posting_list = {}
for doc_no in range(0,len(speeches)):
    speech       = speeches[doc_no]
    clean_speech =  list(set(speech) - set(stopwordtrump_list))
    posting_list = {}
    for word in clean_speech:
        if word.lower() not in stopwordtrump_list:
            word_index = [index for index, value in enumerate(speech) if value == word]
            posting_list[word] = [{doc_no:word_index}]
    for word in posting_list.keys():
        all_posting_list.setdefault(word, []).append(posting_list[word][0])
out_file = open(os.path.join(PICKLES_PATH, "postinglist.pickle"), "wb")
pickle.dump(all_posting_list, out_file)
out_file.close()

### Test for Positional Index & Phrasal Query

In [46]:
query = input('Enter Query : ')

in_file = open(os.path.join(PICKLES_PATH, "postinglist.pickle"), "rb")
posting_list= pickle.load(in_file)
in_file.close()

try:    
    
    query = query.split('/')
    k     = int(query[1]) + 1
    query = query[0]
    query       =  re.sub(r'[^a-zA-Z0-9_\s]+', '', query)
    query_list  = word_tokenize(query)     
    clean_query = [w for w in query_list if w.lower() not in stopwordtrump_list]
    stem_query  = [ps.stem(word) for word in clean_query]
    
    postinglist_1 = posting_list[stem_query[0]]
    postinglist_2 = posting_list[stem_query[1]]
    
    answer = []
    
    while len(postinglist_1) != 0 and len(postinglist_2) != 0:
        doc_1 = list(postinglist_1[0].keys())[0]
        doc_2 = list(postinglist_2[0].keys())[0]
        if doc_1 == doc_2:
            l = []

            postionlist_1 = postinglist_1[0][doc_1]
            postionlist_2 = postinglist_2[0][doc_2]

            while len(postionlist_1) != 0:
                while len(postionlist_2) != 0:
                    if abs(postionlist_1[0] - postionlist_2[0]) == k :
                        l.append(postionlist_2[0])
                    elif postionlist_2[0] > postionlist_1[0]:
                        break
                    postionlist_2.pop(0)
                while len(l) != 0 and abs(l[0] - postionlist_1[0]) > k:
                    l.pop(0)
                for postion in l:
                    answer.append((doc_1,postionlist_1[0],postion))

                postionlist_1.pop(0)

            postinglist_1.pop(0)
            postinglist_2.pop(0)

        elif int(doc_1) < int(doc_2):
            postinglist_1.pop(0)

        else:
            postinglist_2.pop(0)
    print(answer)
    
except (KeyError, ValueError,IndexError):
    print("Wrong format")

Enter Query : ok
Wrong format


In [ ]:
### Test for Inverted Index & Boolean Qu

In [43]:
query = input('Enter Query : ')

in_file = open(os.path.join(PICKLES_PATH, "postinglist.pickle"), "rb")
posting_list= pickle.load(in_file)
in_file.close()


ps = PorterStemmer()
query       =  re.sub(r'[^a-zA-Z0-9_\s]+', '', query)
query       = word_tokenize(query)     

operator = ['NOT','OR','AND','(',')']

postfix_query = GetPostfix(query)
stack = []
try:
    for i in postfix_query:
        if i.upper() not in operator:
            stack.append(i)
        elif i.upper() == 'NOT':
            query_1 = stack.pop()
            if type(query_1) is str:
                query_1  = ps.stem(query_1)
                query_1  = GetPostingList(posting_list[query_1])
            stack.append(NOT(query_1))

        elif i.upper() == 'AND':
            query_1 = stack.pop()
            query_2 = stack.pop()
            if type(query_1) is str:
                query_1  = ps.stem(query_1)
                query_1  = GetPostingList(posting_list[query_1])
            if type(query_2) is str:
                query_2  = ps.stem(query_2)
                query_2  = GetPostingList(posting_list[query_2])
            stack.append(AND(query_1,query_2))

        elif i.upper() == 'OR':
            query_1 = stack.pop()
            query_2 = stack.pop()
            if type(query_1) is str:
                query_1  = ps.stem(query_1)
                query_1  = GetPostingList(posting_list[query_1])
            if type(query_2) is str:
                query_2  = ps.stem(query_2)
                query_2  = GetPostingList(posting_list[query_2])
            stack.append(OR(query_1,query_2))
    answer = stack.pop()
    if type(answer) is str:
        answer  = ps.stem(answer)
        answer  = GetPostingList(posting_list[answer])
    answer.sort()
    print(answer)
except (KeyError ,ValueError):
    print('wrong key words')

Enter Query : hello world
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]


### EXTRA CODE 

In [36]:
# in_file = open(os.path.join(DATASET_PATH, "querylist.txt"), "r")
# query_list = []
# _query = in_file.readlines()
# _query = [q.replace('\n','') for q in _query if q.replace('\n','') != '']
# i = 0
# while 1:
#     if i == len(_query):
#         break
#     query = _query[i]
#     i = i + 1
#     ans   =  _query[i]
#     if ans == 'None':
#         ans = []
#     else:
#         ans = [int(a) for a in ans.split(",")]
#         ans.sort()
#     query_list.append((query,ans))
#     i = i + 1
    
# in_file.close()